In [1]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np
from utils import run_pll
from run_pll import run_dr_raw, run_autognet_raw

def prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree):
    # 1. Simulate or load data
    adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
    print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

    tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
    rho = np.array([[0,0.1,0.2],
                    [0.1,0,0.1],
                    [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
    nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
    gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
    beta = np.array([-1*min_degree,2,-0.2,2,0.1,-2,0.1,2,0.1,0.1])  # shape (10,)

    Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
        burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

    Y_chain = Y_chain[::3]
    A_chain = A_chain[::3]
    L_chain = L_chain[::3]

    res_truth1 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.7,
        R=100,
        burnin_R=100,
        seed=0
    )
    res_truth2 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.3,
        R=100,
        burnin_R=100,
        seed=0
    )

    ground_truth1 = {}
    ground_truth1['average'] = res_truth1['average']
    ground_truth1['direct'] = res_truth1['direct_effect']
    ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
    ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
    ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
    ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
    ground_truth1['psi_zero'] = res_truth1['psi_zero']

    return adj1, Y_chain, A_chain, L_chain, ground_truth1

def run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec=None):
    # 1. Prepare data
    adj1, Y_chain, A_chain, L_chain, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
    
    treat_p = 0.7
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p,
            'psi_0_gamma_only': False,
            'mispec': mispec} for i in range(len(Y_chain))]

    res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    # save results
    np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)


    treat_p = 0.3
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p,
            'psi_0_gamma_only': True,
            'mispec': mispec} for i in range(len(Y_chain))]

    res_list_dr = run_pll(run_dr_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    
    # save results
    np.save(f'run/sim_results/sim_dr_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)


    ##################################################  RUN AUTOGNET #######################################################

    if mispec == 'outcome' or mispec is None:
        if mispec == 'outcome':
            np.random.seed(0)
            L_chain = np.random.binomial(n=1, p=0.2, size=L_chain.shape)

        treat_p = 0.7
        args = [{'Y_chain': Y_chain,
                'A_chain': A_chain,
                'L_chain': L_chain,
                'adj': adj1,
                'i': i,
                'treatment_allocation': treat_p} for i in range(len(Y_chain))]

        res_list_dr = run_pll(run_autognet_raw, args, processes=n_cpu)
        res_list_array_dr = np.array(res_list_dr)
        # save results
        np.save(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)


        treat_p = 0.3
        args = [{'Y_chain': Y_chain,
                'A_chain': A_chain,
                'L_chain': L_chain,
                'adj': adj1,
                'i': i,
                'treatment_allocation': treat_p} for i in range(len(Y_chain))]

        res_list_dr = run_pll(run_autognet_raw, args, processes=n_cpu)
        res_list_array_dr = np.array(res_list_dr)
        # save results
        np.save(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)



In [2]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 2
max_degree = 5

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

2 5


100%|██████████| 200/200 [00:00<00:00, 463.58it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 546.57it/s]


...

100%|██████████| 60/60 [00:00<00:00, 529.50it/s]


Multiprocessing finished.
2 5


100%|██████████| 200/200 [00:00<00:00, 501.46it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [3]:
min_degree = 1
max_degree = 2

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

1 2


100%|██████████| 200/200 [00:00<00:00, 514.12it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 537.43it/s]


...

100%|██████████| 60/60 [00:00<00:00, 542.87it/s]


Multiprocessing finished.
1 2


100%|██████████| 200/200 [00:00<00:00, 520.89it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [4]:
min_degree = 3
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

3 10


100%|██████████| 200/200 [00:00<00:00, 503.70it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 536.05it/s]


...

100%|██████████| 60/60 [00:00<00:00, 544.70it/s]


Multiprocessing finished.
3 10


100%|██████████| 200/200 [00:00<00:00, 498.74it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [5]:
min_degree = 1
max_degree = 4

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

1 4


100%|██████████| 200/200 [00:00<00:00, 512.56it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 545.43it/s]


...

100%|██████████| 60/60 [00:00<00:00, 495.88it/s]

100%|██████████| 60/60 [00:00<00:00, 533.96it/s]


Multiprocessing finished.
1 4


100%|██████████| 200/200 [00:00<00:00, 513.78it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [6]:
min_degree = 1
max_degree = 8

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

1 8


100%|██████████| 200/200 [00:00<00:00, 456.66it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 496.87it/s]


...

100%|██████████| 60/60 [00:00<00:00, 539.65it/s]


Multiprocessing finished.
1 8


100%|██████████| 200/200 [00:00<00:00, 527.40it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [7]:
min_degree = 1
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

1 10


100%|██████████| 200/200 [00:00<00:00, 500.98it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 384.80it/s]

100%|██████████| 60/60 [00:00<00:00, 344.39it/s]

100%|██████████| 60/60 [00:00<00:00, 537.18it/s]


...

100%|██████████| 60/60 [00:00<00:00, 525.50it/s]


Multiprocessing finished.
1 10


100%|██████████| 200/200 [00:00<00:00, 505.74it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [8]:
min_degree = 2
max_degree = 8

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

2 8


100%|██████████| 200/200 [00:00<00:00, 505.79it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 535.30it/s]


...

100%|██████████| 60/60 [00:00<00:00, 514.08it/s]


Multiprocessing finished.
2 8


100%|██████████| 200/200 [00:00<00:00, 508.92it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [9]:
min_degree = 2
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

2 10


100%|██████████| 200/200 [00:00<00:00, 499.80it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


 90%|█████████ | 54/60 [00:00<00:00, 537.58it/s]

...

100%|██████████| 60/60 [00:00<00:00, 503.12it/s]


Multiprocessing finished.
2 10


100%|██████████| 200/200 [00:00<00:00, 500.24it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [10]:
min_degree = 4
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

4 10


100%|██████████| 200/200 [00:00<00:00, 507.07it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 523.45it/s]


...

100%|██████████| 60/60 [00:00<00:00, 535.98it/s]


Multiprocessing finished.
4 10


100%|██████████| 200/200 [00:00<00:00, 531.72it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [11]:
num_sample = 300
num_burn = 20
sample_size = 1600
n_cpu = 10

min_degree = 1
max_degree = 2

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')


min_degree = 2
max_degree = 5

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')


min_degree = 3
max_degree = 10

run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='outcome')
run_sim(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree, mispec='treatment')

1 2


100%|██████████| 200/200 [00:01<00:00, 144.28it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...


Multiprocessing finished.
Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_autognet_raw at 0x15f2a3ac0> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 126.50it/s]


...

Multiprocessing finished.
1 2


100%|██████████| 200/200 [00:02<00:00, 92.91it/s]

Multiprocessing <function run_dr_raw at 0x15f2a39a0> in 100 tasks, with 10 processes...



Process SpawnPoolWorker-982:
Process SpawnPoolWorker-989:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/run_pll.py", line 49, in run_dr_raw
    ret_i = doubly_robust(A_chain[i], L_chain[i], Y_chain[i], adj, treatment_allocation=treatment_allocation, seed=1, return_raw=True,
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/drnet.py", line 205, in doubly_robust
    X_y_eval = build_design_matrix_Y(a_mat[:,i], L_y, Y, adj_matrix)
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/drnet.py", line 38, in build_design_matrix_Y
    Y_nb = get_neighbor_summary(Y.reshape(-1,

KeyboardInterrupt: 